In [1]:
import os
import numpy as np
import pandas as pd


# CSV 2 NPZ
* csv 파일로 구성된 각도 데이터 형태를 갖고와서 .npz 파일 형태로 변환하는 스크립트

## INDEX
1. pandas 라이브러리를 통해 csv 파일을 df 형식으로 불러오기
2. [:3] 은 trans(아바타의 글로벌 3차원 포지션),[3:]은 손을 포함한 joint 회전 값

3. 젠더 = male로 고정(수정해도 됨 female이나...netural이나...)
4. beta는 전부 0으로 고정(이것역시 특정 값이 있다면 수정해도 됨.)
5. frame rate = 60으로 고정(유니티에서 몇 fps로 녹화했는지 확인하기)

6. dmpl은 제외(그래도 돌아감)
7. .npz로 변환

In [4]:
#1. pandas 라이브러리를 통해 csv 파일을 df 형식으로 불러오기


file_name = "moonwalk.csv"


df = pd.read_csv(file_name, header=None,index_col=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,149,150,151,152,153,154,155,156,157,158
0,-0.018713,-0.240441,0.011814,0.189516,0.054596,-0.091360,-0.445250,0.100404,0.076537,-0.424008,...,-5.916267,-1.015726,4.462997,-3.578919,3.991958,0.322437,4.602156,-1.530061,5.894635,0.977758
1,-0.018698,-0.217295,0.013633,0.180802,0.028097,-0.099761,-0.479347,0.113285,0.094218,-0.381547,...,-5.865754,-1.015726,4.462997,-3.578919,4.025781,-0.031842,4.583862,-1.587853,5.885867,1.005560
2,-0.019013,-0.199134,0.014573,0.174213,0.011096,-0.106206,-0.500270,0.115213,0.104910,-0.343219,...,-5.822557,-1.015726,4.462997,-3.578919,4.042985,-0.322867,4.556772,-1.644570,5.874700,1.032748
3,-0.019533,-0.182540,0.015095,0.168626,-0.003109,-0.112202,-0.514724,0.111841,0.112557,-0.292536,...,-5.778657,-1.015726,4.462997,-3.578919,4.050754,-0.632893,4.515877,-1.717885,5.857532,1.067787
4,-0.020116,-0.171114,0.015154,0.165592,-0.012629,-0.116936,-0.522532,0.104611,0.115597,-0.234918,...,-5.744344,-1.015726,4.462997,-3.578919,4.047915,-0.927863,4.465308,-1.805005,5.832995,1.109259


In [3]:
# 해당 셀은 csv에서 로드된 translation값과 pelvis의 루트 회전값을 0으로 초기화 시킴. 필요한 경우에만 사용하기
df.iloc[:,:6] = 0
print(df.iloc[:,:6])

       0    1    2    3    4    5
0    0.0  0.0  0.0  0.0  0.0  0.0
1    0.0  0.0  0.0  0.0  0.0  0.0
2    0.0  0.0  0.0  0.0  0.0  0.0
3    0.0  0.0  0.0  0.0  0.0  0.0
4    0.0  0.0  0.0  0.0  0.0  0.0
..   ...  ...  ...  ...  ...  ...
181  0.0  0.0  0.0  0.0  0.0  0.0
182  0.0  0.0  0.0  0.0  0.0  0.0
183  0.0  0.0  0.0  0.0  0.0  0.0
184  0.0  0.0  0.0  0.0  0.0  0.0
185  0.0  0.0  0.0  0.0  0.0  0.0

[186 rows x 6 columns]


In [5]:
# trans랑 poses 따로 구분해서 변수로 저장
trans = df.iloc[:,0:3]
print(trans.shape)
poses = df.iloc[:,3:]
print(poses.shape)

(186, 3)
(186, 156)


In [6]:
# 필요한 요소들 정렬
'''
gender: male로 고정
mocap_framerate = 60
beta: 왜 16개인진 모르겠지만...
[0,0,0,0,....,0]
'''
gender = 'male'
betas = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
mocap_framerate = 60

save_path = file_name[:-4] + '.npz' # 경로 알아서 바꾸면 됨
print(save_path)
np.savez(save_path,trans =trans, gender = gender, mocap_framerate=mocap_framerate,
                      betas =betas, poses = poses)



moonwalk.npz


In [21]:
# 위의 셀들 한방에 반복문으로 처리할 때의 버전. 에러가 난다면 아마도 대부분 load path나 save path가 맞지 않아서 발생할 확률이 높음 확인 요망
import os
import numpy as np
import pandas as pd

# path setting
src='csv/flight_csv_60f/'
paths = os.listdir(src)
save_path = 'csv2npz/'

for path in paths:
    # file load
    file_name = path
    # print(file_name)
    df = pd.read_csv(os.path.join(src, file_name), header=None,index_col=None)

    # df preprocess
    trans = df.iloc[:,0:3]
    poses = df.iloc[:,3:]


    # const declaration
    gender = 'male'
    betas = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    mocap_framerate = 60 # 나중에 다시 녹화할 때 바꿔야함
    save_name = os.path.join(save_path,file_name[:-4] + '.npz')
    
    np.savez(save_name,trans =trans, gender = gender, mocap_framerate=mocap_framerate,
                      betas =betas, poses = poses)






    
    
